In [68]:
!pip install unidecode

import glob, os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Faradezo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Faradezo\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [41]:
def get_all_files(folder):
    file_list = []
    if os.path.exists(folder):
        for root, dirs, files in os.walk(folder):
            for file in files:
                file_list.append(os.path.join(root,file))
    return file_list

In [42]:
colunistas = get_all_files('textos')

In [47]:
df_col = []
for enu, colunista in enumerate(colunistas):
    print(f'https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/{colunista[7:]}')
    #df_col[enu] = pd.read_json('https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/{colunista[7:]}')
#colunistas
#a[2][7:]

https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/alexandre_garcia.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/arnaldo_jabor.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Carla_Araujo.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Josias_de_Souza.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Juca_Kfouri.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Kennedy_Alencar.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Leonardo_Sakamoto.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Mauricio_Stycer.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Reinaldo_Azevedo.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Ricardo_Kotscho.json
https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/Tales_Faria.js

In [53]:
df_col = pd.read_json('https://raw.githubusercontent.com/Faradezo/Reconhece_autor/main/textos/alexandre_garcia.json')

In [64]:
#Faz a contagem de palavras por cada linha do artigo
df_col['words'] = df_col['texto'].str.split().str.len()

#Elimina qualquer artigo que tenha gerado menos de 100 palavras
df_col = df_col.loc[(df_col['words'] > 100)]

stop = stopwords.words('portuguese')
# stop.append('nao')
stop2 = list()
for word in stop:
  stop2.append(unidecode(word))
  
stemmer = nltk.stem.RSLPStemmer()
# remoção de palavras de parada
df_col['texto_tratado'] = df_col['texto'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#Transforma em minuscula e remoção de acentos 
df_col['texto_tratado'] = df_col['texto_tratado'].str.lower().apply(lambda x: unidecode(x))
# remocação de pontuaçao
df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.punctuation), ' ')
# remocação de numeros
df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.digits), '')
# remoção de palavras de parada (repescagem)
df_col['texto_tratado'] = df_col['texto_tratado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop2)]))

C:\Users\Faradezo\AppData\Local\Temp/ipykernel_7816/1367375677.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.punctuation), ' ')
C:\Users\Faradezo\AppData\Local\Temp/ipykernel_7816/1367375677.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_col['texto_tratado'] = df_col['texto_tratado'].str.replace('[{}]'.format(string.digits), '')


In [69]:
#from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
vectorizer = CountVectorizer()

#Arnaldo Jabor
XCOL = vectorizer.fit_transform(df_col['texto_tratado'])
vocabulary_col = vectorizer.get_feature_names()
pdXCOL = pd.DataFrame(data=XCOL.toarray(), columns=vocabulary_col) #.iloc[:,0::2]

In [71]:
df_XCOL = pdXCOL.T
df_XCOL['total'] = df_XCOL.sum(axis=1) 

# Transpoe a matriz para que as palavras fiquem como linhas, e cada documento como uma linha
df_XCOL = pdXCOL.T

#Calcula a quantidade de vezes que cada palavra é utilizada
df_XCOL['total'] = df_XCOL.sum(axis=1)

#Remove as colunas individuais de cada documento para gerar a tabela
df_XCOL.drop(df_XCOL.columns[0:len(df_col['texto'])], axis=1, inplace=True)

#Ordena o resultado final
df_XCOL = df_XCOL.sort_values(by='total', ascending=False)

#Gera a tabela
df_XCOL.head(30)

,total
tudo,1141
ser,1112
mundo,1022
hoje,972
vida,806
pais,790
anos,735
nada,725
grande,715
brasil,686


In [75]:
n_words = 30

# Transpoe a matriz para que as palavras fiquem como linhas, e cada documento como uma linha
pdXCOLT = pdXCOL.T

#Mantem o percentual de frequencia de uso para comparação
pdXCOLT['presente'] = (pdXCOLT.ne(0).sum(axis=1) -1) / len(df_col['texto'])

#Ordena
pdXCOLT = pdXCOLT.sort_values(by='presente', ascending=False)

#Mantem so as 30 primeiras palavras
pdXCOLT = pdXCOLT.head(n_words)

#mantem 15 textos aleatorios para comparar com os demais exemplos
#sample_drop =  random.sample(range(0, len(df_col['texto'])), len(df_col['texto']) -0)
#pdXCOLT.drop(columns=sample_drop, axis=1, inplace=True)
#pdXCOLT.drop(columns='presente', axis=1, inplace=True)

# Apaga todas as clunas, pois so queremos manter o indice para avaliar os demais textos
pdXCOLT.drop(pdXCOLT.columns[0:424], axis=1, inplace=True)